## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [6]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

experiments = client.search_experiments()

In [11]:
for e in experiments:
    print(e.name, e.experiment_id)

nyc-taxi-experiment 1


In [11]:
# client.create_experiment(name="my-cool-experiment")

'3'

Let's check the latest versions for the experiment with id `1`...

In [14]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=["metrics.rmse ASC"]
)

In [18]:
for run in runs:
    print(f"run name: {run.info.run_name}, run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run name: aged-colt-186, run id: 56c34140806b44698077835bcc30c86a, rmse: 5.8050
run name: useful-stork-660, run id: 71dffff9c1764507bb33cab636feff7d, rmse: 5.8050
run name: casual-dove-807, run id: 2bcbc2b9a7424c59ac38b55982a688b6, rmse: 5.8082
run name: omniscient-mink-424, run id: ba95da70f34a4b2aaf9324b4b237ffe7, rmse: 5.8084
run name: unique-snipe-781, run id: fb29f4e3f1bf482e97108303dd0fab3d, rmse: 5.8095
run name: mysterious-sow-473, run id: 74bb453a7ef7451a947ccd99715b4878, rmse: 5.8107
run name: funny-gnat-488, run id: 598231d0875745fe9c32ca5137b58430, rmse: 5.8153
run name: smiling-stoat-925, run id: ab5b0cda47c84404b4aee08c17d75c8e, rmse: 5.8179
run name: monumental-horse-683, run id: dac90c8673e542c3a1385a552b7541e5, rmse: 5.8216
run name: illustrious-dove-800, run id: c86e666998104578b0a5d4d312be6949, rmse: 5.8227


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [20]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [21]:
run_id = "2bcbc2b9a7424c59ac38b55982a688b6"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

2023/07/07 16:08:47 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/07/07 16:08:47 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/07/07 16:08:47 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1688717327080, current_stage='None', description=None, last_updated_timestamp=1688717327080, name='nyc-taxi-regressor', run_id='2bcbc2b9a7424c59ac38b55982a688b6', run_link=None, source='/mnt/nas/Drive/Dev/zoomcamp/mlops/02-experiment-tracking/mlruns/1/2bcbc2b9a7424c59ac38b55982a688b6/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [36]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Archived
version: 2, stage: Production
version: 3, stage: Staging


In [23]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1688717327080, current_stage='Staging', description=None, last_updated_timestamp=1688717416897, name='nyc-taxi-regressor', run_id='2bcbc2b9a7424c59ac38b55982a688b6', run_link=None, source='/mnt/nas/Drive/Dev/zoomcamp/mlops/02-experiment-tracking/mlruns/1/2bcbc2b9a7424c59ac38b55982a688b6/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [24]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1688717327080, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-07-07', last_updated_timestamp=1688717448854, name='nyc-taxi-regressor', run_id='2bcbc2b9a7424c59ac38b55982a688b6', run_link=None, source='/mnt/nas/Drive/Dev/zoomcamp/mlops/02-experiment-tracking/mlruns/1/2bcbc2b9a7424c59ac38b55982a688b6/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [25]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    # df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    # df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [26]:
df = read_dataframe("data/green_tripdata_2022-03.parquet")

In [28]:
run_id = "56c34140806b44698077835bcc30c86a"
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/tmp/ipykernel_16652/3677731119.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/mnt/nas/Drive/Dev/zoomcamp/mlops/02-experiment-tracking/preprocessor'

In [29]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

/home/william/.pyenv/versions/3.10.12/envs/zoomcamp-exp-track/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [30]:
X_test = preprocess(df, dv)

In [31]:
target = "duration"
y_test = df[target].values

In [33]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/07/07 16:14:55 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 139 ms, sys: 5.29 ms, total: 144 ms
Wall time: 300 ms


{'rmse': 6.245888027515617}

In [38]:
%time test_model(name=model_name, stage="None", X_test=X_test, y_test=y_test)

2023/07/07 16:23:27 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 16.1 ms, sys: 4.94 ms, total: 21 ms
Wall time: 176 ms


{'rmse': 298.8807911245536}

In [39]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Archived",
    # archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1688718181536, current_stage='Archived', description='', last_updated_timestamp=1688718247996, name='nyc-taxi-regressor', run_id='efb32d803c80430c84de3548cc266d06', run_link='', source='/mnt/nas/Drive/Dev/zoomcamp/mlops/02-experiment-tracking/mlruns/1/efb32d803c80430c84de3548cc266d06/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>